# Coffee shop in Brussels


### Background

I would like to know where all the coffee shops in the center of Brussels are and where would be a good place to start a nex coffee based business. To solve this question we will use data from the FourSquare API. This data can tell us where all the coffee shops are situated and where might already be an abundance of shops. By looking for hot spots, we can find a fitting place where there aren't a lot of other options for the customers. This will be the location where I can start my business.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


In [5]:
import folium 
from folium import plugins

In [6]:
CLIENT_ID = 'E2QA1ZHI25QVTCULVRSOXHH2R0DA553UVMCSXHZA40EFCTBV' # your Foursquare ID
CLIENT_SECRET = 'KMGEI4SHEP1KEPOIQ2GH4OEJISJ3FWSKBJMWGUKQQKMUWEKU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 150
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: E2QA1ZHI25QVTCULVRSOXHH2R0DA553UVMCSXHZA40EFCTBV
CLIENT_SECRET:KMGEI4SHEP1KEPOIQ2GH4OEJISJ3FWSKBJMWGUKQQKMUWEKU


In [7]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Brussels",
    "radius": 3000,
    "limit": 150}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [8]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [9]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Brussels', 'Brussels')

In [10]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 50.8692106356402, 'lng': 4.391129820111508},
  'sw': {'lat': 50.824044978634575, 'lng': 4.3341367885148125}},
 150)

In [11]:
d["geocode"]

{'what': '',
 'where': 'brussels',
 'center': {'lat': 50.85045, 'lng': 4.34878},
 'displayString': 'Brussels, Belgium',
 'cc': 'BE',
 'geometry': {'bounds': {'ne': {'lat': 50.91418299917369,
    'lng': 4.480998926270525},
   'sw': {'lat': 50.76358303321909, 'lng': 4.244593105665788}}},
 'slug': 'brussels-belgium',
 'longId': '72057594040728802'}

In [12]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [13]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [14]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5419723a498ebf995790a376',
  'name': 'Chicago Café',
  'location': {'address': 'Vlaamsesteenweg 45 Rue de Flandre',
   'lat': 50.85160499425515,
   'lng': 4.346479661199551,
   'labeledLatLngs': [{'label': 'display',
     'lat': 50.85160499425515,
     'lng': 4.346479661199551}],
   'postalCode': '1000',
   'cc': 'BE',
   'city': 'Brussel',
   'state': 'Brussels Hoofdstedelijk Gewest',
   'country': 'België',
   'formattedAddress': ['Vlaamsesteenweg 45 Rue de Flandre',
    '1000 Brussel',
    'België']},
  'categories': [{'id': '4bf58dd8d48988d143941735',
    'name': 'Breakfast Spot',
    'pluralName': 'Breakfast Spots',
    'shortName': 'Breakfast',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []},
  'venuePag

In [15]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': 'Vlaamsesteenweg 45 Rue de Flandre', 'lat': 50.85160499425515, 'lng': 4.346479661199551, 'labeledLatLngs': [{'label': 'display', 'lat': 50.85160499425515, 'lng': 4.346479661199551}], 'postalCode': '1000', 'cc': 'BE', 'city': 'Brussel', 'state': 'Brussels Hoofdstedelijk Gewest', 'country': 'België', 'formattedAddress': ['Vlaamsesteenweg 45 Rue de Flandre', '1000 Brussel', 'België']}
{'address': 'Rue du Marché aux Poulets 49', 'lat': 50.8485, 'lng': 4.35109, 'labeledLatLngs': [{'label': 'display', 'lat': 50.8485, 'lng': 4.35109}], 'postalCode': '1000', 'cc': 'BE', 'neighborhood': 'Brussels', 'city': 'Brussel', 'state': 'Brussels Hoofdstedelijk Gewest', 'country': 'België', 'formattedAddress': ['Rue du Marché aux Poulets 49', '1000 Brussel', 'België']}
{'address': 'Kartuizersstraat 4 Rue des Chartreux', 'lat': 50.84882547763037, 'lng': 4.347332361256083, 'labeledLatLngs': [{'label': 'display', 'lat': 50.84882547763037, 'lng': 4.347332361256083}], 'postalCode': '1000', 'cc': 'B

,uid,name,shortname,address,postalcode,lat,lng
0,5419723a498ebf995790a376,Chicago Café,Breakfast,,1000,50.851605,4.346480
1,4ce657f50f196dcb65b336ae,Corica Grand Place,Coffee Shop,,1000,50.848500,4.351090
2,4b3f3290f964a52097a425e3,A.M. Sweet,Tea Room,,1000,50.848825,4.347332
3,57a19405498e118a09a3f31e,MOK Specialty Coffee Roastery & Bar,Coffee Shop,,1000,50.852877,4.341740
4,4c012a5594c1c9b60acdba5f,Moka - Coffee & Beer,Coffee Shop,,1000,50.846732,4.347617


In [16]:
Brussels_center = d["geocode"]["center"]
Brussels_center

{'lat': 50.85045, 'lng': 4.34878}

In [17]:
map_Brussels = folium.Map(location=[50.85045, 4.34878], zoom_start=15)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Brussels)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Brussels.add_child(plugins.HeatMap(hm_data))

map_Brussels

In [18]:
lat = 50.85045
lng = 4.34878
map_Brussels = folium.Map(location=[lat, lng], zoom_start=12)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Rent this lcoation!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Brussels)
map_Brussels